In [1]:
import cv2
import numpy as np
import os
import sys
sys.path.append(os.path.abspath('../'))
import pandas as pd
import datetime
import random
from utilities.helper_functions import *
import config
import matplotlib.pyplot as plt

In [2]:
## Imports array of videos and names of videos from a folder
# @param path: path to folder containing videos
# Returns a list of videos and a list of names
def load_videos(path):
    videos = []
    names = []
    for file in os.listdir(path):
        if file.endswith('.mp4'):
            names.append(path.split('/')[-1] + file.split('.')[0])
            video_capture = cv2.VideoCapture(path + '/' + file)
            video = []
            while True:
                ret, frame = video_capture.read()
                if not ret:
                    break
                # convert to grayscale
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                video.append(frame)
            videos.append(np.array(video))
    return videos, names


In [3]:
video_paths = [config.video_output_path + trial for trial in os.listdir(config.video_output_path) ]
videos, names = load_videos(video_paths[0])

In [4]:
## Take each a video and then subtract each frame from the next frame to get the next movement
def calculate_movement(video):
    movement = []
    for i in range(len(video) - 1):
        movement.append(video[i + 1].astype('int16') - video[i].astype('int16'))
    return np.abs((np.array(movement) + 256) // 2).astype('uint8')


In [7]:
def extract_movement_frames(video):
    movement = []
    for i in range(len(video) - 1):
        movement.append(np.abs(np.sum(video[i + 1].astype('int16') - video[i].astype('int16'))))
    # take a moving average of the movement
    try:
        movement = np.convolve(movement, np.ones(100) / 100, mode='same')
    except (ValueError):
        movement = np.zeros(len(movement))
    return movement


In [8]:
# iterate through all the videos, extract interactions, and then save the video of the interactions to a folder
for path in video_paths:
    videos, names = load_videos(path)
    for video, name in zip(videos, names):
        movement = extract_movement_frames(video)
        if np.max(movement) > 1000 and len(movement) > 1:
            # find the first frame where the movement is greater than 1000
            start = np.where(movement > 1000)[0][0]
            # find the last frame where the movement is greater than 1000
            end = np.where(movement > 1000)[0][-1]
            movemnent_extracted_video = video[start:end]
            # save the video to the movement_extracted folder
            helper_functions.save_video(movemnent_extracted_video, config.movement_extracted_output_path + name + '.mp4')

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
move_vid = calculate_movement(videos[0])

In [ ]:
# show vid in a window using opencv
for vid in videos:
    move_vid = calculate_movement(vid)
    i = 0
    while not i == len(move_vid):
        
        # calculate net movement by summing the absolute value of the difference between each frame
        net_move = np.sum(move_vid[i].astype(np.int32) - 128)
        display_frame = move_vid[i].copy()
        # add text to the frame with net move
        cv2.putText(display_frame, str(abs(net_move) > 1000), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Video', display_frame)
        i += 1
        if cv2.waitKey(12) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

KeyboardInterrupt: 